<a href="https://colab.research.google.com/github/rimbourouphael/NSY107/blob/main/LSTM_Balanced_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Long-Short Term Memory (LSTM) Network

This code is part our research on malware detection and classification using Deep Learning and Deep Graph Convolutional Neural Networks.

For more information or citation, please refer to our research paper:

"Oliveira, Angelo; Sassi, Renato José (2019): Behavioral Malware Detection Using Deep Graph Convolutional Neural Networks. TechRxiv. Preprint." at https://doi.org/10.36227/techrxiv.10043099.v1

For the dataset, please refer to our repository:

https://ieee-dataport.org/open-access/malware-analysis-datasets-api-call-sequences

#### Balanced Dataset

In [1]:
!pip install skorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 155 kB 5.8 MB/s 


In [2]:
import numpy as np
SEED = 137
np.random.seed(SEED)

import time
from collections import Counter

import pandas as pd

from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix
from sklearn.metrics import precision_score, f1_score, roc_auc_score

from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

import torch
torch.manual_seed(SEED)
import torch.nn as nn
import torch.nn.functional as F

from skorch.classifier import NeuralNetBinaryClassifier

### Data Preprocessing

In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/rimbourouphael/NSY107/main/dynamic_api_call_sequence_per_malware_100_0_306.csv')
df.head()

,hash,t_0,t_1,t_2,t_3,t_4,t_5,t_6,t_7,t_8,...,t_91,t_92,t_93,t_94,t_95,t_96,t_97,t_98,t_99,malware
0,071e8c3f8922e186e57548cd4c703a5d,112,274,158,215,274,158,215,298,76,...,71,297,135,171,215,35,208,56,71,1
1,33f8e6d08a6aae939f25a8e0d63dd523,82,208,187,208,172,117,172,117,172,...,81,240,117,71,297,135,171,215,35,1
2,b68abd064e975e1c6d5f25e748663076,16,110,240,117,240,117,240,117,240,...,65,112,123,65,112,123,65,113,112,1
3,72049be7bd30ea61297ea624ae198067,82,208,187,208,172,117,172,117,172,...,208,302,208,302,187,208,302,228,302,1
4,c9b3700a77facf29172f32df6bc77f48,82,240,117,240,117,240,117,240,117,...,209,260,40,209,260,141,260,141,260,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43876 entries, 0 to 43875
Columns: 102 entries, hash to malware
dtypes: int64(101), object(1)
memory usage: 34.1+ MB


In [5]:
X = df.drop(['hash', 'malware'], axis = 1).values.astype(int)
y = df['malware'].values.astype(int)
print(X.shape)
print(y.shape)

(43876, 100)
(43876,)


In [6]:
print(X.min())
print(X.max())

0
306


In [7]:
def check_imbalance(dataset):
    count = sorted(Counter(dataset).items())
    print(count)
    print(count[1][1] / count[0][1])
    return

In [8]:
random_undersampler = RandomUnderSampler(random_state = SEED)

X, y = random_undersampler.fit_resample(X, y)

check_imbalance(y)

[(0, 1079), (1, 1079)]
1.0


In [9]:
check_imbalance(y)

[(0, 1079), (1, 1079)]
1.0


In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = SEED)

In [11]:
check_imbalance(y_train)
check_imbalance(y_test)

del df, X, y

[(0, 738), (1, 772)]
1.046070460704607
[(0, 341), (1, 307)]
0.9002932551319648


### Architecture

In [12]:
class LSTM_network(nn.Module):
    
    def __init__(self, input_dim, hidden_dim, dropout_rate):
        
        super(LSTM_network, self).__init__()
        
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.dropout_rate = dropout_rate
        
        self.lstm = nn.LSTM(self.input_dim, self.hidden_dim, batch_first = True)
        self.dropout = nn.Dropout(p = self.dropout_rate)
        self.fc = nn.Linear(self.hidden_dim, 1)
        
    def forward(self, X):
        
        X = F.one_hot(X, num_classes = self.input_dim).float().cuda()
        
        # Hidden layer shape: (num_layers, batch_size, hidden_dim)
        hidden_0 = (torch.zeros(1, X.size(0), self.hidden_dim).float().cuda(),
                    torch.zeros(1, X.size(0), self.hidden_dim).float().cuda())
        
        # Input/Output shape: (batch_size, seq_len, input_dim)
        _, self.hidden = self.lstm(X, hidden_0)
                
        H = self.hidden[0].squeeze()
        H = self.dropout(H)
        H = self.fc(H)
                
        return H.squeeze()

model = LSTM_network(
    input_dim = 307,
    hidden_dim = 10,
    dropout_rate = 0.4
)

print(model)
print(f'\nParameters: {np.sum([param.numel() for param in model.parameters()])}')
del model

LSTM_network(
  (lstm): LSTM(307, 10, batch_first=True)
  (dropout): Dropout(p=0.4, inplace=False)
  (fc): Linear(in_features=10, out_features=1, bias=True)
)

Parameters: 12771


### Hyperparameter Optimization

In [ ]:
start_training = time.time()

torch.manual_seed(SEED)

input_dim = 307

net = NeuralNetBinaryClassifier(
    LSTM_network,
    module__input_dim = input_dim,
    module__hidden_dim = 10,
    module__dropout_rate = 0.4,
    batch_size = 32,
    max_epochs = 10,
    train_split = None,
    optimizer = torch.optim.Adam,
    iterator_train__shuffle = True,
    device = 'cuda'
)

pipe = Pipeline([
    ('net', net)
])

params = {
    'net__module__hidden_dim' : [10, 40, 70, 100],
    'net__module__dropout_rate' : [0.4, 0.5, 0.6],
    'net__batch_size' : [32, 64, 128],
    'net__max_epochs' : [10, 20, 30]
}

gs = GridSearchCV(
    pipe,
    params,
    refit = False,
    cv = StratifiedKFold(n_splits = 5, random_state = SEED, shuffle = True),
    scoring = lambda net_gs, X_gs, y_gs : roc_auc_score(y_gs, net_gs.predict_proba(X_gs)),
    verbose = 2
)

gs.fit(X_train, y_train.astype(np.float))

print(gs.best_score_, gs.best_params_)

end_training = (time.time() - start_training) / 60

print(f'\nDone! Time: {end_training:.2f} min')

#### Best configuration:

0.9404987294237913 {'net__batch_size': 32, 'net__max_epochs': 30, 'net__module__dropout_rate': 0.4, 'net__module__hidden_dim': 10}

### Training

In [13]:
torch.manual_seed(SEED)

# hidden_dim = gs.best_params_['net__module__hidden_dim']
# dropout_rate = gs.best_params_['net__module__dropout_rate']
# batch_size = gs.best_params_['net__batch_size']
# max_epochs = gs.best_params_['net__max_epochs']

input_dim = 307
hidden_dim = 10
dropout_rate = 0.4
batch_size = 32
max_epochs = 30

net = NeuralNetBinaryClassifier(
    LSTM_network,
    module__input_dim = input_dim,
    module__hidden_dim = hidden_dim,
    module__dropout_rate = dropout_rate,
    batch_size = batch_size,
    max_epochs = max_epochs,
    train_split = None,
    optimizer = torch.optim.Adam,
    iterator_train__shuffle = True,
    device = 'cuda'
)

pipe = Pipeline([
    ('net', net)
])

pipe.fit(X_train, y_train.astype(np.float))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


  epoch    train_loss     dur
-------  ------------  ------
      1        0.6375  0.4213
      2        0.4992  0.2219
      3        0.4429  0.2156
      4        0.3841  0.2256
      5        0.3472  0.2203
      6        0.3430  0.2141
      7        0.3276  0.2143
      8        0.2930  0.2199
      9        0.2713  0.2174
     10        0.2465  0.2130
     11        0.2506  0.2162
     12        0.2339  0.2153
     13        0.2135  0.2190
     14        0.2005  0.2092
     15        0.2538  0.2144
     16        0.2314  0.2158
     17        0.1983  0.2113
     18        0.1773  0.2158
     19        0.1519  0.2196
     20        0.1398  0.2175
     21        0.1226  0.2127
     22        0.1997  0.2199
     23        0.2163  0.2163
     24        0.1494  0.2160
     25        0.1211  0.2141
     26        0.1205  0.2110
     27        0.0936  0.2157
     28        0.1026  0.2098
     29        0.1076  0.2109
     30        0.1030  0.2149


Pipeline(steps=[('net',
                 <class 'skorch.classifier.NeuralNetBinaryClassifier'>[initialized](
  module_=LSTM_network(
    (lstm): LSTM(307, 10, batch_first=True)
    (dropout): Dropout(p=0.4, inplace=False)
    (fc): Linear(in_features=10, out_features=1, bias=True)
  ),
))])

### Evaluation

In [14]:
def model_evaluate(y, pred):
    print('Confusion matrix\n[TN FP]\n[FN TP]')
    print(confusion_matrix(y >= 0.5, pred >= 0.5))
    print(f'Accuracy: {accuracy_score(y >= 0.5, pred >= 0.5):.4f}')
    print(f'Precision: {precision_score(y >= 0.5, pred >= 0.5):.4f}')
    print(f'Recall: {recall_score(y >= 0.5, pred >= 0.5):.4f}')
    print(f'F1-Score: {f1_score(y >= 0.5, pred >= 0.5):.4f}')
    print(f'ROC AUC: {roc_auc_score(y, pred):.4f}')
    return

In [15]:
model_evaluate(y_test, np.ones(len(y_test)))

Confusion matrix
[TN FP]
[FN TP]
[[  0 341]
 [  0 307]]
Accuracy: 0.4738
Precision: 0.4738
Recall: 1.0000
F1-Score: 0.6429
ROC AUC: 0.5000


In [16]:
X_test_1 = pipe.predict_proba(X_test)[:, 1]
model_evaluate(y_test, X_test_1)

Confusion matrix
[TN FP]
[FN TP]
[[306  35]
 [ 44 263]]
Accuracy: 0.8781
Precision: 0.8826
Recall: 0.8567
F1-Score: 0.8694
ROC AUC: 0.9449


In [ ]:
%%javascript
IPython.notebook.save_notebook()

<IPython.core.display.Javascript object>